In [1]:
! git clone https://github.com/stdereka/knowledge-distillation
! wget https://s3.amazonaws.com/fast-ai-imageclas/imagewoof2-320.tgz
! tar zxf imagewoof2-320.tgz

Cloning into 'knowledge-distillation'...
remote: Enumerating objects: 32, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 32 (delta 8), reused 26 (delta 6), pack-reused 0
Unpacking objects: 100% (32/32), done.
--2020-09-17 20:43:27--  https://s3.amazonaws.com/fast-ai-imageclas/imagewoof2-320.tgz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.48.238
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.48.238|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 328288506 (313M) [application/x-tar]
Saving to: ‘imagewoof2-320.tgz’

imagewoof2-320.tgz  100%[===================>] 313.08M  77.0MB/s    in 4.1s    

2020-09-17 20:43:31 (77.1 MB/s) - ‘imagewoof2-320.tgz’ saved [328288506/328288506]



In [2]:
import sys
sys.path.insert(1, "/content/knowledge-distillation")

In [10]:
import torch
import random
import numpy as np
from pathlib import Path
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from torch.utils.data import DataLoader

In [4]:
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

train_on_gpu = torch.cuda.is_available()
if not train_on_gpu:
    print('Training on CPU')
    DEVICE = torch.device("cpu")
else:
    print('Training on GPU')
    DEVICE = torch.device("cuda")

SEED = 666
seed_everything(SEED)

TRAIN_DIR = Path('./imagewoof2-320/train')
TEST_DIR = Path('./imagewoof2-320/val')

train_val_files = sorted(list(TRAIN_DIR.rglob('*.JPEG')))
test_files = sorted(list(TEST_DIR.rglob('*.JPEG')))

train_val_labels = [path.parent.name for path in train_val_files]
LABEL_ENCODER = LabelEncoder()
LABEL_ENCODER.fit(train_val_labels)

Training on GPU


LabelEncoder()

In [5]:
from datasets import Imagewoof
from models import resnet18

In [6]:
train_dataset = Imagewoof(train_val_files, LABEL_ENCODER)
test_dataset = Imagewoof(test_files, LABEL_ENCODER)

In [7]:
resnet = resnet18(10, DEVICE)

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth


In [8]:
from teacher_train import train, predict
history_15 = train(train_dataset, test_dataset, resnet, 15, 64, DEVICE)

epoch:   7%|▋         | 1/15 [01:00<14:04, 60.30s/it]

train_loss: 3.5795, val_loss: 2.3024, train_acc: 0.1169, val_acc: 0.1036


epoch:  13%|█▎        | 2/15 [02:00<13:03, 60.25s/it]

train_loss: 2.1240, val_loss: 1.5425, train_acc: 0.2062, val_acc: 0.3535


epoch:  20%|██        | 3/15 [03:00<12:01, 60.12s/it]

...

train_loss: 0.1050, val_loss: 0.3462, train_acc: 0.9672, val_acc: 0.9023


epoch:  93%|█████████▎| 14/15 [13:58<00:59, 59.73s/it]

train_loss: 0.1035, val_loss: 0.3352, train_acc: 0.9649, val_acc: 0.9079


epoch: 100%|██████████| 15/15 [14:57<00:00, 59.82s/it]

train_loss: 0.0912, val_loss: 0.2847, train_acc: 0.9703, val_acc: 0.9193


In [11]:
temperatures = [1, 2, 3, 5, 8, 10]
cv = StratifiedKFold(n_splits=4, shuffle=True, random_state=SEED)
ground = np.array(Imagewoof(train_val_files, LABEL_ENCODER).labels)
oof_probas = [np.empty((len(train_val_files), 10), float) for t in temperatures]

for fold, (train_idx, val_idx) in enumerate(cv.split(train_val_files, LABEL_ENCODER.transform(train_val_labels))):
    print('Training on fold', fold + 1)

    val_dataset = Imagewoof(np.array(train_val_files)[val_idx], LABEL_ENCODER)
    train_dataset = Imagewoof(np.array(train_val_files)[train_idx], LABEL_ENCODER)
    
    model = resnet18(10, DEVICE)
    history = train(train_dataset, val_dataset, model=model, epochs=7, batch_size=64, device=DEVICE)
    
    val_loader = DataLoader(val_dataset, shuffle=False, batch_size=64)
    labels_val = ground[val_idx]

    for i, temp in enumerate(temperatures):
        probas_val = predict(model, val_loader, DEVICE, temp)
        oof_probas[i][val_idx] = probas_val
        preds_val = np.argmax(probas_val, axis=1)
    print(f'Fold {fold + 1} accuracy score:', accuracy_score(labels_val, preds_val))

oof_preds = np.argmax(oof_probas[0], axis=1)
print('OOF accuracy score:', accuracy_score(ground, oof_preds))

Training on fold 1


epoch:  14%|█▍        | 1/7 [00:41<04:10, 41.75s/it]

train_loss: 3.9897, val_loss: 2.3016, train_acc: 0.1124, val_acc: 0.1360


epoch:  29%|██▊       | 2/7 [01:23<03:28, 41.66s/it]

train_loss: 2.2725, val_loss: 2.1274, train_acc: 0.1333, val_acc: 0.1812


...

train_loss: 0.2968, val_loss: 0.3159, train_acc: 0.9041, val_acc: 0.8927


epoch:  86%|████████▌ | 6/7 [04:09<00:41, 41.62s/it]

train_loss: 0.2331, val_loss: 0.2845, train_acc: 0.9275, val_acc: 0.9020


epoch: 100%|██████████| 7/7 [04:51<00:00, 41.62s/it]


train_loss: 0.1998, val_loss: 0.2818, train_acc: 0.9372, val_acc: 0.9043
Fold 4 accuracy score: 0.9042553191489362
OOF accuracy score: 0.9184487534626039


In [12]:
import os
os.makedirs("./dark_knowledge", exist_ok=True)

for i in range(len(oof_probas)):
    np.save(f"./dark_knowledge/resnet18_T_{temperatures[i]}.npy", oof_probas[i])